In [17]:
import numpy as np

## jupyter notebook showing how to inject events with genie
This is fairly slow, and can be cpu intensive, so don't simulate more than 1,000 events like this. For large datasets use slurm with the py script

#### also the following cell was just to resolve some annoying import problems I got with vs code's conda environment

In [18]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

2025-05-07 00:17:06,555 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-05-07 00:17:06,555 - h5py._conv - DEBUG - Creating converter from 5 to 7
2025-05-07 00:17:06,556 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-05-07 00:17:06,556 - h5py._conv - DEBUG - Creating converter from 5 to 7


Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [19]:
from prometheus import Prometheus, config

In [20]:



import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final
from gevgen_injection import run_gevgen, convert_to_gst, convert_to_gtrac

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


In [21]:

def main(num_events, simset=1):
    # timing
    start_time = time.time()


    print(f"Using simset: {simset}")
    processing_start_time = time.time()



    logger.info("Starting GENIE simulation")
    genie_start = time.time()
    ghep_file = run_gevgen(num_events, OUTPUT_DIR)
    gtrac_file = convert_to_gtrac(ghep_file, OUTPUT_DIR)
    
    genie_time = time.time() - genie_start
    logger.info(f"GENIE simulation completed in {genie_time:.2f} seconds")
    prometheus_set, primary_set = parse_and_convert_genie(gtrac_file)
    processing_end_time = time.time()
    print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
    prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

    # inject into cylinder:
    primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                        prometheus_set, ## child particle information
                        cylinder_radius=500, ## meters
                        cylinder_height = 1000, # meters
                        cylinder_center = (0, 0, 0), # meters not including offset
                        )

    primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


    # toming for serialization and saving
    save_start_time = time.time()
    # position arrays to serialized strings
    prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
    num_events = len(prometheus_set)
    print(f"Processing {num_events} events")
    prometheus_set.to_parquet(prometheus_file_path)
    primary_set.to_parquet(primrary_file_path)
    save_end_time = time.time()
    print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

    ## GENIE stuff:
    config["injection"]["name"] = "GENIE"
    config["run"]["outfile"] = f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"
    config["run"]["nevents"] = num_events
    config["injection"]["GENIE"] = config["injection"].get("GENIE", {})
    config["injection"]["GENIE"]["paths"] = config["injection"]["GENIE"].get("paths", {})
    config["injection"]["GENIE"]["inject"] = False ## ideally this should toggle using gevgen and be true, but for now setting this to false helps solve a headache in promtheus.py
    config["injection"]["GENIE"]["simulation"] = {}
    ## geofile:
    config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
    ## ppc configuration:
    config['photon propagator']['name'] = 'PPC_UPGRADE'
  #  config["photon propagator"]["PPC_UPGRADE"] = config["photon propagator"].get("PPC_UPGRADE", {})
  #  config["photon propagator"]["PPC_UPGRADE"]["paths"] = config["photon propagator"]["PPC_UPGRADE"].get("paths", {})
   # config["photon propagator"]["PPC"]["paths"]["ppctables"] = f"{RESOURCE_DIR}/PPC_tables/spx" ## spx for test

    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
    
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
    print(config["photon propagator"])
    config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = False ## just for printing!

    # Timing for Prometheus simulation
    sim_start_time = time.time()
    p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
    p.sim()
    sim_end_time = time.time()
    print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")
    
    # End timing
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print('Finished without catastrophic error')
    return




In [22]:
num_events = 10 ## just a short dataset for testing
simset = 11
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")
print("Launching simulation")
main(num_events, simset)
print("Finished call")
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

2025-05-07 00:17:10,124 - __main__ - INFO - Starting GENIE simulation
2025-05-07 00:17:10,125 - gevgen_injection - INFO - Running GEVGEN with 10 events
2025-05-07 00:17:10,125 - gevgen_injection - INFO - genie command: ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '10', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '1,100', '-f', 'x^-1', '--seed', '12345', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-generator-list', 'Default', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


--------------------------------------------------------------
--------------------------------------------------------------
Launching simulation
Using simset: 11
cmd:  ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '10', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '1,100', '-f', 'x^-1', '--seed', '12345', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-generator-list', 'Default', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


2025-05-07 00:17:12,940 - gevgen_injection - INFO - GEVGEN completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root
2025-05-07 00:17:12,941 - gevgen_injection - INFO - Converting to GTRAC format using rootracker




*****************************************************************************************************************
*                                                                                                               *
*                                                   .oooooo.    oooooooooooo ooooo      ooo ooooo oooooooooooo  *
*                                                  d8P'  `Y8b   `888'     `8 `888b.     `8' `888' `888'     `8  *
*                                                 888            888          8 `88b.    8   888   888          * 
*                        Ndyooym          dN      888            888oooo8     8   `88b.  8   888   888oooo8     *
*                     Nds//+sdmoy       d+m       888     ooooo  888    "     8     `88b.8   888   888    "     * 
*                   Nh+//ohN  m+s      N//syyN    `88.    .88'   888       o  8       `888   888   888       o  *
*                 Ny+//od   Nh+oN       o///+      `Y8bood8P'   o888ooooood8 o8o    

2025-05-07 00:17:13,931 - gevgen_injection - INFO - GTRAC conversion completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 00:17:13,932 - __main__ - INFO - GENIE simulation completed in 3.81 seconds
2025-05-07 00:17:13,932 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 00:17:13,933 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 00:17:13,938 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 00:17:13,947 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 00:17:13,965 -

...

Handling charged pion/kaon 211
returning hadron 2112


Configuring ppc in "./ppc_tmpdir11/"
Configuring nextgen sensors in "./ppc_tmpdir11/"
Configuring icemodel in "./ppc_tmpdir11/"
Configuring tiltmodel in "./ppc_tmpdir11/"
Configuring holeice from "./ppc_tmpdir11/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 9666  hits: 0

Handling charged pion/kaon 211
Handling charged pion/kaon 211


Configuring ppc in "./ppc_tmpdir11/"
Configuring nextgen sensors in "./ppc_tmpdir11/"
Configuring icemodel in "./ppc_tmpdir11/"
Configuring tiltmodel in "./ppc_tmpdir11/"
Configuring holeice from "./ppc_tmpdir11/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 1751  hits: 0

Handling charged pion/kaon -211
Prometheus simulation completed in 26.16 seconds
Total execution time: 30.08 seconds (0.50 minutes)
Finished without catastrophic error
I am melting.... AHHHHHH!!!!
Finished call
--------------------------------------------------------------
--------------------------------------------------------------


In [23]:

primary_set_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_set_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
out_parquet =  f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"

prometheus_set = pd.read_parquet(prometheus_set_path)
primary_set = pd.read_parquet(primary_set_path)
out_parquet = pd.read_parquet(out_parquet)

### the position on the primary set and prometheus set don't have the ~1950m z offset btw

In [24]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,True,50.867880,14,CC,2.605261,4.211347,-1,-1,-125.779411,-473.966507,126.823457,"[-125.77941111623578, -473.96650663434957, 126...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
1,True,37.561409,14,NC,2.374384,5.934769,-1,-1,-459.816789,8.817938,440.059548,"[-459.81678925284564, 8.817937807986805, 440.0...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[N...
2,True,18.655406,14,CC,1.139310,0.100646,-1,-1,-419.017652,-131.425969,-437.097368,"[-419.01765229079376, -131.42596853716427, -43...",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],RES;..."
3,True,53.072486,14,NC,2.316795,5.539670,-1,-1,-190.971042,299.240846,16.016785,"[-190.9710415621579, 299.24084615050396, 16.01...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[N...
4,True,84.073415,14,NC,1.433237,1.412667,-1,-1,366.220733,310.848403,-110.767335,"[366.2207329867429, 310.8484031526923, -110.76...",-1,nu:14;tgt:1000080160;N:2212;q:2(v);proc:Weak[N...
5,True,5.574043,14,CC,1.719728,5.893192,-1,-1,470.816226,40.832938,-351.228272,"[470.81622562671515, 40.83293847599876, -351.2...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[CC],RES;..."
6,True,8.386735,14,NC,0.785183,1.488576,-1,-1,-168.234808,-378.582446,-478.167656,"[-168.23480778420554, -378.5824464733994, -478...",-1,"nu:14;tgt:1000010010;N:2212;proc:Weak[NC],RES;..."
7,True,74.359844,14,CC,1.976330,3.358825,-1,-1,-387.955065,89.428704,274.452204,"[-387.9550646468035, 89.42870441118039, 274.45...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
8,True,35.221388,14,CC,1.875701,3.339287,-1,-1,-54.015452,-68.699051,160.318433,"[-54.01545178994569, -68.69905051045404, 160.3...",-1,nu:14;tgt:1000010010;N:2212;q:-2(s);proc:Weak[...
9,True,57.079045,14,NC,2.025456,4.999265,-1,-1,-24.137726,-213.903741,-391.510061,"[-24.13772587037426, -213.9037412699296, -391....",-1,nu:14;tgt:1000010010;N:2212;q:1(v);proc:Weak[N...


In [25]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, Fal...","[0.00632, 26.519930189224535, 8.20249092481366...","[22, 13, 111, 211, 111, 2112, 111, 111, 111, 2...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[1.735044785131836, 2.5662752420152004, 2.5763...","[2.2229536006855937, -1.993807140463022, -2.23...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-125.77941111623578, -125.77941111623578, -12...","[-473.96650663434957, -473.96650663434957, -47...","[126.82345711879645, 126.82345711879645, 126.8...","[[-125.77941111623578, -473.96650663434957, 12...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
1,"[False, False, False, False, False, False, False]","[18.982163553833917, 15.396966329320206, 1.152...","[14, -211, 2212, 2212, 2112, 111, 2212]","[NC, NC, NC, NC, NC, NC, NC]","[2.2097612599313106, 2.5069730306156615, 2.569...","[-0.3862834708877146, -0.19344145766592682, 0....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-459.81678925284564, -459.81678925284564, -45...","[8.817937807986805, 8.817937807986805, 8.81793...","[440.059547546334, 440.059547546334, 440.05954...","[[-459.81678925284564, 8.817937807986805, 440....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
2,"[False, False, False, False, False, False]","[15.60294420105663, 1.0416240060075583, 0.2962...","[13, 2212, 211, -211, 2112, 111]","[CC, CC, CC, CC, CC, CC]","[1.149080709284613, 0.49396699207717565, 1.835...","[0.13091134039353808, 0.634562489187677, 0.098...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-419.01765229079376, -419.01765229079376, -41...","[-131.42596853716427, -131.42596853716427, -13...","[-437.09736777620077, -437.09736777620077, -43...","[[-419.01765229079376, -131.42596853716427, -4...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
3,"[False, False, False, False]","[0.00618, 46.26659256248036, 4.720359315294123...","[22, 14, 2212, -211]","[NC, NC, NC, NC]","[0.5182711956274971, 2.3402416393588714, 2.046...","[-0.2713889451496981, -0.6687815353035361, -0....","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[-190.9710415621579, -190.9710415621579, -190....","[299.24084615050396, 299.24084615050396, 299.2...","[16.016784599395464, 16.016784599395464, 16.01...","[[-190.9710415621579, 299.24084615050396, 16.0...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
4,"[False, False, False, False, False, False, Fal...","[0.00673, 2.97400063941656, 6.7640063681574984...","[22, 14, 111, 111, 321, 111, -311, 111, -211, ...","[NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, N...","[1.8607813079622328, 1.5897603344405866, 1.450...","[0.06931525394694649, 1.2624618503812741, 1.43...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[366.2207329867429, 366.2207329867429, 366.220...","[310.8484031526923, 310.8484031526923, 310.848...","[-110.767334888199, -110.767334888199, -110.76...","[[366.2207329867429, 310.8484031526923, -110.7...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
5,"[False, False, False, False, False, False]","[0.00632, 4.905321299347469, 1.097888905933379...","[22, 13, 2212, 2112, 211, 211]","[CC, CC, CC, CC, CC, CC]","[1.6480076079761263, 1.741613579680293, 1.3897...","[0.4101234868924203, -0.31797683242360375, -0....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[470.81622562671515, 470.81622562671515, 470.8...","[40.83293847599876, 40.83293847599876, 40.8329...","[-351.22827165033584, -351.22827165033584, -35...","[[470.81622562671515, 40.83293847599876, -351....","[-1.0, -1.0, 

### and the normal mc_truth and photons fields from the parquet:


In [26]:
out_parquet

,mc_truth,photons
0,"{'interaction': 1, 'initial_state_energy': 50....","{'sensor_pos_x': [-132.8, -132.8, -132.8], 'se..."
1,"{'interaction': 2, 'initial_state_energy': 37....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
2,"{'interaction': 1, 'initial_state_energy': 18....","{'sensor_pos_x': [-358.44, -447.74], 'sensor_p..."
3,"{'interaction': 2, 'initial_state_energy': 53....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
4,"{'interaction': 2, 'initial_state_energy': 84....","{'sensor_pos_x': [382.35, 382.35], 'sensor_pos..."
5,"{'interaction': 1, 'initial_state_energy': 5.5...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
6,"{'interaction': 2, 'initial_state_energy': 8.3...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
7,"{'interaction': 1, 'initial_state_energy': 74....","{'sensor_pos_x': [-358.44, -481.6], 'sensor_po..."
8,"{'interaction': 1, 'initial_state_energy': 35....","{'sensor_pos_x': [-9.68, -77.8, -77.8], 'senso..."
9,"{'interaction': 2, 'initial_state_energy': 57....","{'sensor_pos_x': [-43.27], 'sensor_pos_y': [-2..."


In [30]:
out_parquet['mc_truth'][0]

{'interaction': 1,
 'initial_state_energy': 50.86788025887283,
 'initial_state_type': 14,
 'initial_state_zenith': 2.6052610903628572,
 'initial_state_azimuth': -2.0718382521983134,
 'initial_state_x': -119.90858165887153,
 'initial_state_y': -476.48511516148443,
 'initial_state_z': -1845.1523083850861,
 'final_state_energy': array([6.32000000e-03, 2.65199302e+01, 1.42527101e-02, 4.12656699e-02,
        5.01399945e-02, 8.20249092e+00, 4.78939147e-01, 2.97010665e+00,
        3.27583806e+00, 4.34336996e+00, 1.98125695e+00, 3.13401853e+00,
        3.17927121e-01, 9.40051250e-01, 5.74838298e-01]),
 'final_state_type': array([  22.,   13.,   11.,   14.,  -12.,  111.,  211.,  111., 2112.,
         111.,  111.,  111.,  211., 2212.,  111.]),
 'final_state_zenith': array([1.73504479, 2.56627524, 1.38551193, 2.13226019, 1.17465952,
        2.57634904, 2.35618355, 2.49262838, 2.54746185, 2.5760178 ,
        2.59539191, 2.59652847, 2.43170624, 0.50111065, 2.42007116]),
 'final_state_azimuth': arra

In [29]:
out_parquet['photons'][0]

{'sensor_pos_x': array([-132.8, -132.8, -132.8]),
 'sensor_pos_y': array([-501.45, -501.45, -501.45]),
 'sensor_pos_z': array([-1890.53, -1873.51, -1924.58]),
 'string_id': array([2, 2, 2]),
 'sensor_id': array([27, 26, 29]),
 't': array([263.849762, 420.466553, 562.347412]),
 'id_idx': array([2, 2, 2])}

#### so I guess you don't need to load the prometheus and primary set if you have the mc_truth